## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_trimmed = application_df.copy()
application_df_trimmed = application_df_trimmed.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df_trimmed.nunique(axis=0)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

### Function to reduce categories within a column based on df, column, and count limit

In [4]:
def replace_with_other(df, column, count):
    
    # Get value counts
    val_count_series = df[column].value_counts()
    
    # create a list of items to be replaced for values less than a givent count
    items_to_replace = []
    for i in range(len(val_count_series)):
        if val_count_series[i] < count:
            items_to_replace.append(val_count_series.index[i])

    # Replace in dataframe
    for item in items_to_replace:
        df[column] = df[column].replace(item,"Other")
    
    return df

## Filter for one application type

In [5]:
application_df_trimmed['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Filter for a single application type and then drop that column
application_df_filtered = application_df_trimmed[application_df_trimmed['APPLICATION_TYPE'] == 'T3']
application_df_filtered = application_df_filtered.drop(['APPLICATION_TYPE'], axis=1)
application_df_filtered.head()

,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
1,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1


## Additional Feature Reduction
There are 27,037 rows. The next few sections review each column distribution for further reduction.<br>
A rough guideline is to keep 97.5% feature differentiation per column (26,361 rows)

In [7]:
application_df_filtered['CLASSIFICATION'].value_counts()

C1000    12909
C2000     5189
C1200     4825
C2100     1883
C7000      777
         ...  
C2170        1
C4120        1
C2561        1
C1245        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# 97% (26,230) rows captured by counts 500+
# Reduce CLASSIFICATION to items with >= 500 counts
application_df_filtered = replace_with_other(application_df_filtered, 'CLASSIFICATION', 500)
application_df_filtered['CLASSIFICATION'].value_counts()

C1000    12909
C2000     5189
C1200     4825
C2100     1883
Other     1454
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
application_df_filtered['AFFILIATION'].value_counts()

Independent         15934
CompanySponsored    11006
Family/Parent          56
National               28
Regional               10
Other                   3
Name: AFFILIATION, dtype: int64

In [10]:
# 99.6% (26,940) rows capture by counts 10,000+
# Reduce AFFILIATION to items with >= 10000 counts
application_df_filtered = replace_with_other(application_df_filtered, 'AFFILIATION', 10000)
application_df_filtered['AFFILIATION'].value_counts()

Independent         15934
CompanySponsored    11006
Other                  97
Name: AFFILIATION, dtype: int64

In [11]:
application_df_filtered['ORGANIZATION'].value_counts()
# 98% are captured by counts 5,000+ but there are only 2 other categories so leaving alone

Trust           21174
Association      5538
Co-operative      289
Corporation        36
Name: ORGANIZATION, dtype: int64

In [12]:
application_df_filtered['INCOME_AMT'].value_counts()

0                18625
25000-99999       3138
100000-499999     2949
1M-5M              842
1-9999             563
10000-24999        443
10M-50M            198
5M-10M             153
50M+               126
Name: INCOME_AMT, dtype: int64

In [13]:
# 91% are in the top 3 buckets with the million dollar ranges making up most of the tail
# Consolidate, just the million dollar ranges

# create the list to be replaced for values less than 500
millions_to_replace = ['1M-5M', '5M-10M', '10M-50M', '50M+']

# Replace in dataframe
for item in millions_to_replace:
    application_df_filtered['INCOME_AMT'] = application_df_filtered['INCOME_AMT'].replace(item,"1M+")

application_df_filtered['INCOME_AMT'].value_counts()

0                18625
25000-99999       3138
100000-499999     2949
1M+               1319
1-9999             563
10000-24999        443
Name: INCOME_AMT, dtype: int64

In [14]:
application_df_filtered['USE_CASE'].value_counts()

Preservation     26469
CommunityServ      297
Heathcare          146
ProductDev         123
Other                2
Name: USE_CASE, dtype: int64

In [15]:
application_df_filtered['STATUS'].value_counts()

1    27032
0        5
Name: STATUS, dtype: int64

In [16]:
application_df_filtered['SPECIAL_CONSIDERATIONS'].value_counts()

N    27010
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [17]:
# 97.9% (26469/27037) T3 rows have Use Case = Preservation
# 99.98% (27032/27037) T3 rows have Status = 1
# 99.9% (27010/27037) T3 rows have Special Consideration = N

# Drop these columns
application_df_filtered = application_df_filtered.drop(['USE_CASE', 'STATUS', 'SPECIAL_CONSIDERATIONS'], axis=1)
application_df_filtered.head()

,AFFILIATION,CLASSIFICATION,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
1,Independent,C2000,Co-operative,1-9999,108590,1
3,CompanySponsored,C2000,Trust,10000-24999,6692,1
4,Independent,C1000,Trust,100000-499999,142590,1
5,Independent,C1200,Trust,0,5000,1
6,Independent,C1000,Trust,100000-499999,31452,1


## Convert categorical data to numerical

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
# (AFFILIATION,CLASSIFICATION, USE_CASE, ORGANIZATION)
application_df_noncat = pd.get_dummies(data=application_df_filtered, columns=['AFFILIATION', 'CLASSIFICATION', 
                                                                             'ORGANIZATION', 'INCOME_AMT'])
application_df_noncat.head()

,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C7000,...,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M+,INCOME_AMT_25000-99999
1,108590,1,0,1,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,6692,1,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
4,142590,1,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
5,5000,1,0,1,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
6,31452,1,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [19]:
# Split our preprocessed data into our features and target arrays
y = application_df_noncat['IS_SUCCESSFUL']
X = application_df_noncat.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=4, activation="tanh", input_dim=20))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 84        
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 89 (356.00 Byte)
Trainable params: 89 (356.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Compile the model 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Set a callback for every 5 epochs to an HDF5 file
mc = tf.keras.callbacks.ModelCheckpoint(filepath='AlphabetSoupCharity_Optimization3_fit.h5', save_weights_only=True, save_freq=5)

# Train the model over 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[mc])

Epoch 1/50
634/634 [==============================] - 2s 2ms/step - loss: 0.6254 - accuracy: 0.6571
Epoch 2/50
634/634 [==============================] - 1s 2ms/step - loss: 0.5856 - accuracy: 0.7117
Epoch 3/50
634/634 [==============================] - 1s 2ms/step - loss: 0.5801 - accuracy: 0.7181
Epoch 4/50
634/634 [==============================] - 1s 2ms/step - loss: 0.5770 - accuracy: 0.7182
Epoch 5/50
634/634 [==============================] - 1s 2ms/step - loss: 0.5747 - accuracy: 0.7185
Epoch 6/50
634/634 [==============================] - 1s 2ms/step - loss: 0.5729 - accuracy: 0.7194
Epoch 7/50
634/634 [==============================] - 1s 2ms/step - loss: 0.5716 - accuracy: 0.7199
Epoch 8/50
634/634 [==============================] - 1s 2ms/step - loss: 0.5706 - accuracy: 0.7187
Epoch 9/50
634/634 [==============================] - 1s 2ms/step - loss: 0.5696 - accuracy: 0.7196
Epoch 10/50
634/634 [==============================] - 1s 2ms/step - loss: 0.5693 - accuracy: 0.7191

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

212/212 - 0s - loss: 0.5639 - accuracy: 0.7217 - 245ms/epoch - 1ms/step
Loss: 0.5638517737388611, Accuracy: 0.7217455506324768


In [ ]:
# Export the model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization3_model.h5', save_format='h5')